# Simple quantum circuits

## Prerequisite: installation of myqlm

    pip install myqlm


## Documentation

You will find code examples on https://myqlm.github.io.

## 1. Bell state circuit

Our aim is to prepare the Bell state $$\frac{|00\rangle + |11\rangle}{\sqrt{2}}$$

### Solution

In [ ]:
from qat.lang.AQASM import H, CNOT, Program

prog = Program()
reg = prog.qalloc(2)

prog.apply(H, reg[0])
prog.apply(CNOT, reg)

circ = prog.to_circ()

%qatdisplay circ

We now execute this circuit on a Quantum Processing Unit (QPU):

In [ ]:
from qat.qpus import get_default_qpu
qpu = get_default_qpu()

job = circ.to_job(nbshots=10, aggregate_data=False)
res = qpu.submit(job)

The number of shots is the number of independent final $Z$-axis measurements on the quantum state. Thus, we expect to see the 10 outcomes of measuring both qubits on the $Z$ axis. 

We print the results:

In [ ]:
for sample in res:
    print(sample.state)

We can tell the QPU to aggregate the data, i.e collect the statistics of the outcomes to compute the histogram of the shots, and hence the estimated probability (``sample.probability``) (with its statistical error ``sample.err``) of a computational state in the final distribution:

In [ ]:
job = circ.to_job(nbshots=10)
res = qpu.submit(job)
for sample in res:
    print(sample.state, sample.probability, sample.err)

Finally, since we are performing classical simulation, we have access to the exact probabilities, and to the probability amplitudes of the states (because we are doing pure-state simulations at this stage). This is achieved by choosing an infinite number of shots, which we choose, by convention, by setting ``nbshots=0``:

In [ ]:
job = circ.to_job(nbshots=0)
res = qpu.submit(job)
for sample in res:
    print(sample.state, sample.probability, sample.amplitude)

## 2. A simple variational algorithm

Here we study how to compute the (approximate) ground state energy of a Hamiltonian, here $$H = X_0 X_1 + Y_0 Y_1 + Z_0 Z_1.$$



In [ ]:
from qat.core import Observable as Obs, Term
from qat.lang import Program, RY, CNOT
from qat.qpus import get_default_qpu
from qat.plugins import ScipyMinimizePlugin

# we instantiate the Hamiltonian we want to approximate the ground state energy of
hamiltonian = (
    Obs.sigma_z(0) * Obs.sigma_z(1)
    + Obs.sigma_x(0) * Obs.sigma_x(1)
    + Obs.sigma_y(0) * Obs.sigma_y(1)
)

Our variational circuit is very simple, it has only two parameters:

In [ ]:
# we construct the variational circuit (ansatz)
prog = Program()
reg = prog.qalloc(2)
thetas = [prog.new_var(float, '\\theta_%s'%i) for i in range(2)]
RY(thetas[0])(reg[0])
RY(thetas[1])(reg[1])
CNOT(reg[0], reg[1])
var_circ = prog.to_circ()

var_circ.display()

In [ ]:
# construct a (variational) job with the variational circuit and the observable
job = var_circ.to_job(observable=hamiltonian)

# we now build a stack that can handle variational jobs
qpu = get_default_qpu()
optimizer_scipy = ScipyMinimizePlugin(method="COBYLA",
                                        tol=1e-6,
                                        options={"maxiter": 200},
                                        x0=[0, 0])
stack = optimizer_scipy | qpu

# we submit the job and print the optimized variational energy (the exact GS energy is -3)
result = stack.submit(job)
# the output of the optimizer can be found here
print(result.meta_data['optimizer_data'])
print(f"Minimum VQE energy = {result.value}")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(eval(result.meta_data['optimization_trace']))
plt.xlabel("optimization step")
plt.ylabel("energy");

In [ ]:
# what is the final state?
# we replace the variables by their optimal values

angles = eval(result.meta_data["parameter_map"])
print(angles)

circ = var_circ(**angles)
circ.display()

In [ ]:
# we run the corresponding circuit

qpu = get_default_qpu()
res = qpu.submit(circ.to_job()) # this time the job has no observable, to obtain samples only
for sample in res:
    print(sample.state, sample.probability)

Comment on this result.